In [1]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from IPython.display import display, Image, HTML
import numpy as np
import time
import model_vgg
import pandas as pd
import re

/home/shkim/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
DATA_DIR = "../data/"
TEST_DIR = DATA_DIR + "test_resize/"

test_images =  [TEST_DIR+i for i in os.listdir(TEST_DIR)]

print("test_images: ", len(test_images))

test_images.sort(key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
print(test_images[:10])

test_images:  12500
['../data/test_resize/1.jpg', '../data/test_resize/2.jpg', '../data/test_resize/3.jpg', '../data/test_resize/4.jpg', '../data/test_resize/5.jpg', '../data/test_resize/6.jpg', '../data/test_resize/7.jpg', '../data/test_resize/8.jpg', '../data/test_resize/9.jpg', '../data/test_resize/10.jpg']


In [3]:
sess = tf.Session()
model = model_vgg.Model_Vgg19(sess, "model")
model.build_net(150)

Instructions for updating:
Use the retry module or similar alternatives.


In [4]:
batch_size = 20

sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [5]:
ckpt_path = saver.restore(sess, "./model/model.ckpt")

INFO:tensorflow:Restoring parameters from ./model/model.ckpt


In [6]:
#test_images = test_images[:100]

start_time = time.time()

test_count = len(test_images)
total_batch = int(test_count / batch_size)

probability_list = []
for i in range(total_batch):
    batch_files, batch_x_image = model_vgg.getBatchData_x(test_images, batch_size, i)
    batch_p = model.probability(batch_x_image)
    probability_list.extend(batch_p)
    #print(batch_files)
    #print(batch_p)
    #df.append({"filename": batch_files, "dog": batch_p[:, 0], "cat": batch_p[:, 1]}, ignore_index=True)
    #df.append({"filename": batch_files}, ignore_index=True)

    
print("--- %.2f seconds ---" %(time.time() - start_time))

--- 84.83 seconds ---


In [7]:
hp = np.array(probability_list)
df = pd.DataFrame({"id": range(1, len(probability_list) + 1), "label": hp[:, 0]})
df.to_csv('submission.csv', index=False)